In [164]:
# Import the basic spark library

from pyspark.sql import SparkSession

# Create an entry point to the PySpark Application
spark = SparkSession.builder \
      .master("local") \
      .appName("schemaBook") \
      .getOrCreate()
# master contains the URL of your remote spark instance or 'local'

In [165]:
#schema www
df_www= spark.read.option("multiline", "true").json("www-db-manual.json")
df_www.printSchema()
df_www.show()


root
 |-- author: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- key: string (nullable = true)
 |-- note: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- url: array (nullable = true)
 |    |-- element: string (containsNull = true)

+--------------------+--------------------+--------------------+--------------------+
|              author|                 key|                note|                 url|
+--------------------+--------------------+--------------------+--------------------+
|[{Alejandro Rodri...|  homepages/189/9749|[Western Norway U...|[https://orcid.or...|
|[{Kikuo Fujimura,...|homepages/f/Kikuo...|[Ohio State Unive...|[http://www.cse.o...|
|[{Pierfrancesco F...|homepages/f/Pierf...|                null|[http://garga.iet...|
|[{Farshad Fotouhi...|homepages/f/Farsh...|[Wayne State Univ...|[http://www.cs.wa...|
|[{Ada Wai-Che

In [166]:
#import
from pyspark.sql.types import StructType,StructField, StringType, ArrayType

2. VERSION 1 Add a new WEBSITE
The following command can be used to insert a website for Barbara Hausmann, affiliating her to the Technical University of Graz, Austria .

In [167]:
data = [([("Barbara Hausmann","Barbara Hausmann")], "NewWebsiteKey", ["Technical University of Graz, Austria"], ["www.barbarahausmann.com"])]

schema= StructType([
      StructField("author",ArrayType(StructType([
       StructField("orcid", StringType(), True),
       StructField("name", StringType(), True)
       ])),True),
      StructField("key",StringType(),True),
      StructField("note",ArrayType(StringType()), True),
      StructField("url",ArrayType(StringType()), True)
      ])
newWeb= spark.createDataFrame(data = data, schema = schema)
newWeb.printSchema()
newWeb.show(truncate=False)

root
 |-- author: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- orcid: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- key: string (nullable = true)
 |-- note: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- url: array (nullable = true)
 |    |-- element: string (containsNull = true)

+--------------------------------------+-------------+---------------------------------------+-------------------------+
|author                                |key          |note                                   |url                      |
+--------------------------------------+-------------+---------------------------------------+-------------------------+
|[{Barbara Hausmann, Barbara Hausmann}]|NewWebsiteKey|[Technical University of Graz, Austria]|[www.barbarahausmann.com]|
+--------------------------------------+-------------+---------------------------------------+-------------------------+



In [168]:
df_www =df_www.unionByName(newWeb, allowMissingColumns=True)
df_www.printSchema()
df_www.show(truncate=False)

root
 |-- author: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- key: string (nullable = true)
 |-- note: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- url: array (nullable = true)
 |    |-- element: string (containsNull = true)

+----------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [169]:
#check
from pyspark.sql.functions import array_contains, col
df_www.filter(array_contains(df_www.note, "Technical University of Graz, Austria")).select(col("key"),col("note")).show(truncate=False)

+---------------------+------------------------------------------------------------------------------------------------------------------------------------------------+
|key                  |note                                                                                                                                            |
+---------------------+------------------------------------------------------------------------------------------------------------------------------------------------+
|homepages/f/DWFellner|[Fraunhofer Institute for Computer Graphics Research (IGD), TU Darmstadt, Department of Computer Science, Technical University of Graz, Austria]|
|NewWebsiteKey        |[Technical University of Graz, Austria]                                                                                                         |
+---------------------+------------------------------------------------------------------------------------------------------------------------------------